In [1]:
%%time
!pip uninstall -y torch
!pip install -q --no-index --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-vllm vllm
!pip install -q -U --upgrade /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -q -U --upgrade /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl
!pip install -q --no-deps --no-index /kaggle/input/hf-libraries/sentence-transformers/sentence_transformers-3.1.0-py3-none-any.whl

Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
CPU times: user 2.06 s, sys: 555 ms, total: 2.61 s
Wall time: 3min 29s


In [2]:
%%capture
# !pip install --no-index  /kaggle/input/trans-pip/transformers-4.46.2-py3-none-any.whl --find-links=/kaggle/input/trans-pip
# !pip install --no-index /kaggle/input/trans-pip/tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --find-links=/kaggle/input/trans-pip

!pip install transformers peft accelerate \
    -q -U --no-index --find-links /kaggle/input/lmsys-wheel-files

In [3]:
%%capture
!pip install --no-index /kaggle/input/bitsandbytes0-42-0/bitsandbytes-0.42.0-py3-none-any.whl --find-links=/kaggle/input/bitsandbytes0-42-0
!pip install --no-index  /kaggle/input/bitsandbytes0-42-0/optimum-1.21.2-py3-none-any.whl --find-links=/kaggle/input/bitsandbytes0-42-0
!pip install --no-index  /kaggle/input/bitsandbytes0-42-0/auto_gptq-0.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --find-links=/kaggle/input/bitsandbytes0-42-0

In [4]:
!pip install /kaggle/input/trans-pip/tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/trans-pip/transformers-4.46.2-py3-none-any.whl

Processing /kaggle/input/trans-pip/tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.42.4 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.20.3 which is incompatible.
Processing /kaggle/input/trans-pip/transformers-4.46.2-py3-none-any.whl
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optimum 1

In [5]:
from tqdm.auto import tqdm
# from bs4 import BeautifulSoup
import gc
import pandas as pd
import pickle
import sys
import numpy as np
from tqdm.autonotebook import trange
from sklearn.model_selection import GroupKFold
import json
import torch
from numpy.linalg import norm
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel,BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
)
from peft import PeftModel
from peft import prepare_model_for_kbit_training
import json
import copy
import warnings
warnings.filterwarnings('ignore')
import os
import re

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

os.environ["TOKENIZERS_PARALLELISM"] = "false"


def apk(actual, predicted, k=25):
    """
    Computes the average precision at k.
    
    This function computes the average prescision at k between two lists of
    items.
    
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    
    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=25):
    """
    Computes the mean average precision at k.
    
    This function computes the mean average prescision at k between two lists
    of lists of items.
    
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

def batch_to_device(batch, target_device):
    """
    send a pytorch batch to a device (CPU/GPU)
    """
    for key in batch:
        if isinstance(batch[key], Tensor):
            batch[key] = batch[key].to(target_device)
    return batch

def last_token_pool(last_hidden_states: Tensor,
                    attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

def inference(df, model, tokenizer, device):
    batch_size = 2
    max_length = 512
    sentences = list(df['query_text'].values)
    pids = list(df['order_index'].values)
    all_embeddings = []
    length_sorted_idx = np.argsort([-len(sen) for sen in sentences])
    sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
    for start_index in trange(0, len(sentences), batch_size, desc="Batches", disable=False):
        sentences_batch = sentences_sorted[start_index: start_index + batch_size]
        features = tokenizer(sentences_batch, max_length=max_length, padding=True, truncation=True,
                             return_tensors="pt")
        features = batch_to_device(features, device)
        with torch.no_grad():
            outputs = model(**features)
            embeddings = last_token_pool(outputs.last_hidden_state, features['attention_mask'])
            embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
            embeddings = embeddings.detach().cpu().numpy().tolist()
        all_embeddings.extend(embeddings)

    all_embeddings = [np.array(all_embeddings[idx]).reshape(1, -1) for idx in np.argsort(length_sorted_idx)]

    sentence_embeddings = np.concatenate(all_embeddings, axis=0)
    result = {pids[i]: em for i, em in enumerate(sentence_embeddings)}
    return result

In [6]:
def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

# task_description = 'Given a math question and a misconcepte incorrect answer, retrieve the most accurate explanation for the misconception.'



task_description = 'Given a math question with correct answer and a misconcepted incorrect answer, retrieve the most accurate misconception for the incorrect answer.'

path_prefix = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"


tra = pd.read_csv(f"{path_prefix}/test.csv")



misconception_mapping = pd.read_csv(f"{path_prefix}/misconception_mapping.csv")


first_model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"
tokenizer = AutoTokenizer.from_pretrained(first_model_path)

def preprocess_text(x):
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x



PROMPT  = """Here is a question about {ConstructName} ({SubjectName}).
Question: {Question}
Correct Answer: {CorrectAnswer}
Incorrect Answer: {IncorrectAnswer}

Generate a brief rationale for the Incorrect Answer in a sentences, within 50 tokens. Describe the reasoning process that might lead to this choice, including any logical errors or misconceptions.

Keep the explanation concise and focused on the key points, and ensure it does not exceed 50 tokens
***Important***
1.There is no need to analyze the relationship between the wrong answer and the correct answer. Simply describe the rationale of the incorrect answer in a few words.
2.There is no need to have fields such as "Incorrect Answer:" and "Incorrect Answer is". Just give the rationale directly.
3.Output results in English.
"""


def apply_template(row, tokenizer, targetCol):
    messages = [
        {
            "role": "user", 
            "content": preprocess_text(
                PROMPT.format(
                    ConstructName=row["ConstructName"],
                    SubjectName=row["SubjectName"],
                    Question=row["QuestionText"],
                    IncorrectAnswer=row[f"Answer{targetCol}Text"],
                    CorrectAnswer=row[f"Answer{row.CorrectAnswer}Text"],
                )
            )
        }
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text



train_data = []
for _,row in tra.iterrows():
    for c in ['A','B','C','D']:
        if c ==row['CorrectAnswer']:
            continue
        if f'Answer{c}Text' not in row:
            continue
        real_answer_id = row['CorrectAnswer']
        real_text = row[f'Answer{real_answer_id}Text']
        # query_text =f"###question###:{row['SubjectName']}-{row['ConstructName']}-{row['QuestionText']}\n###Correct Answer###:{real_text}\n###Misconcepte Incorrect answer###:{c}.{row[f'Answer{c}Text']}"
        query_text = f"### SubjectName: {row['SubjectName']}\n### ConstructName: {row['ConstructName']}\n### Question: {row['QuestionText']}\n### Correct Answer: {real_text}\n### Misconcepte Incorrect answer: {row[f'Answer{c}Text']}"
        row['query_text'] = get_detailed_instruct(task_description,query_text)
        row['answer_name'] = c
        # row['input_llm_text'] = apply_template(row, tokenizer, c)
        # row['input_llm_text'] = apply_template(tokenizer, query_text)
        row['input_llm_text'] = apply_template(row, tokenizer, c)
        row['answer_name'] = c ### add
        train_data.append(copy.deepcopy(row))
train_df = pd.DataFrame(train_data)
train_df['order_index'] = list(range(len(train_df)))


train_df.to_parquet("train_df.parquet", index=False)

import gc
del train_df, tra, tokenizer
gc.collect()

print("ok")

ok


In [7]:
%%writefile run_vllm_for_rationale.py

import re
import vllm
import pandas as pd

train_df = pd.read_parquet("train_df.parquet")
first_model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"


llm = vllm.LLM(
    first_model_path,
    quantization="awq",
    tensor_parallel_size=2,
    gpu_memory_utilization=0.90, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=5120,
    disable_log_stats=True
)
tokenizer = llm.get_tokenizer()


responses = llm.generate(
    train_df["input_llm_text"].values,
    vllm.SamplingParams(
        n=1,  # Number of output sequences to return for each prompt.
        top_p=0.8,  # Float that controls the cumulative probability of the top tokens to consider.
        temperature=0,  # randomness of the sampling
        seed=777, # Seed for reprodicibility
        skip_special_tokens=False,  # Whether to skip special tokens in the output.
        max_tokens=256,  # Maximum number of tokens to generate per output sequence.
    ),
    use_tqdm=True
)

responses = [x.outputs[0].text for x in responses]
train_df["rationale"] = responses
train_df['query_text'] = train_df['query_text'] + "\n### Rationale: " + train_df['rationale']


train_df.to_parquet("tmp.parquet", index=False)

Writing run_vllm_for_rationale.py


In [8]:
!python run_vllm_for_rationale.py

WARNING 12-12 11:24:38 config.py:246] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 12-12 11:24:38 config.py:715] Defaulting to use mp for distributed inference
INFO 12-12 11:24:38 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1', speculative_config=None, tokenizer='/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=5120, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig

In [9]:
train_df = pd.read_parquet("tmp.parquet")

In [10]:
train_df

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,query_text,answer_name,input_llm_text,order_index,rationale
0,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,Instruct: Given a math question with correct a...,B,"<|im_start|>system\nYou are Qwen, created by A...",0,Adding brackets around 4-5 keeps subtraction i...
1,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,Instruct: Given a math question with correct a...,C,"<|im_start|>system\nYou are Qwen, created by A...",1,"Mistakenly grouping all numbers in brackets, i..."
2,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,Instruct: Given a math question with correct a...,D,"<|im_start|>system\nYou are Qwen, created by A...",2,Believes no brackets are needed as operations ...
3,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,Instruct: Given a math question with correct a...,A,"<|im_start|>system\nYou are Qwen, created by A...",3,One might factor the numerator as (m+3)(m-1) a...
4,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,Instruct: Given a math question with correct a...,B,"<|im_start|>system\nYou are Qwen, created by A...",4,One might incorrectly factor the numerator as ...
5,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,Instruct: Given a math question with correct a...,C,"<|im_start|>system\nYou are Qwen, created by A...",5,One might factor the numerator as (m-1)(m+3) a...
6,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,Instruct: Given a math question with correct a...,A,"<|im_start|>system\nYou are Qwen, created by A...",6,"Cutting plants in half would halve the range, ..."
7,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,Instruct: Given a math question with correct a...,C,"<|im_start|>system\nYou are Qwen, created by A...",7,One might think both are correct if they misun...
8,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,Instruct: Given a math question with correct a...,D,"<|im_start|>system\nYou are Qwen, created by A...",8,Both actions change the data set. Halving the ...


In [11]:
%%writefile fold1.py
from tqdm.auto import tqdm
# from bs4 import BeautifulSoup
import gc
import pandas as pd
import pickle
import sys
import numpy as np
from tqdm.autonotebook import trange
from sklearn.model_selection import GroupKFold
import json
import torch
from numpy.linalg import norm
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel,BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
)
from peft import PeftModel
from peft import prepare_model_for_kbit_training
import json
import copy
import warnings
warnings.filterwarnings('ignore')
import os
import re

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


fold = 1
train_df = pd.read_parquet("tmp.parquet")

def batch_to_device(batch, target_device):
    """
    send a pytorch batch to a device (CPU/GPU)
    """
    for key in batch:
        if isinstance(batch[key], Tensor):
            batch[key] = batch[key].to(target_device)
    return batch

def last_token_pool(last_hidden_states: Tensor,
                    attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'
def inference(df, model, tokenizer, device):
    batch_size = 2
    max_length = 512
    sentences = list(df['query_text'].values)
    pids = list(df['order_index'].values)
    all_embeddings = []
    length_sorted_idx = np.argsort([-len(sen) for sen in sentences])
    sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
    for start_index in trange(0, len(sentences), batch_size, desc="Batches", disable=False):
        sentences_batch = sentences_sorted[start_index: start_index + batch_size]
        features = tokenizer(sentences_batch, max_length=max_length, padding=True, truncation=True,
                             return_tensors="pt")
        features = batch_to_device(features, device)
        with torch.no_grad():
            outputs = model(**features)
            embeddings = last_token_pool(outputs.last_hidden_state, features['attention_mask'])
            embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
            embeddings = embeddings.detach().cpu().numpy().tolist()
        all_embeddings.extend(embeddings)

    all_embeddings = [np.array(all_embeddings[idx]).reshape(1, -1) for idx in np.argsort(length_sorted_idx)]

    sentence_embeddings = np.concatenate(all_embeddings, axis=0)
    result = {int(pids[i]): em for i, em in enumerate(sentence_embeddings)}
    return result

path_prefix = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"
model_path = "/kaggle/input/qwen-instruct-14b"
lora_path="/kaggle/input/aa1201-1815/hp_code_64_128_epoch5_tmp002/epoch_4_model" #529
# lora_path = "/kaggle/input/v8v13v14-19/v8v13v14_19/lora_merge.bin"
# device='cuda'
VALID = False

misconception_mapping = pd.read_csv(f"{path_prefix}/misconception_mapping.csv")

tokenizer = AutoTokenizer.from_pretrained(model_path)
# bnb_config = BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_quant_type="nf4",
#             bnb_4bit_compute_dtype=torch.bfloat16
#         )
model = AutoModel.from_pretrained(model_path, 
                                  torch_dtype=torch.float16,
                                  # quantization_config=bnb_config,
                                  device_map='auto',
                                 )
if lora_path!="none":

    model = PeftModel.from_pretrained(model, lora_path)#.eval().to(args.device)
    # model = model.merge_and_unload()
model = model.eval()

train_embeddings = inference(train_df, model, tokenizer, 'cuda')

misconception_mapping['query_text'] = misconception_mapping['MisconceptionName']
misconception_mapping['order_index'] = misconception_mapping['MisconceptionId']
doc_embeddings = inference(misconception_mapping, model, tokenizer, 'cuda')

sentence_embeddings = np.concatenate([e.reshape(1, -1) for e in list(doc_embeddings.values())])
index_text_embeddings_index = {index: paper_id for index, paper_id in
                                         enumerate(list(doc_embeddings.keys()))}


cosine_similarities = []
for _, row in tqdm(train_df.iterrows()):
    query_id = row['order_index']
    query_em = train_embeddings[query_id].reshape(1, -1)
    
    cosine_similarity = np.dot(query_em, sentence_embeddings.T).flatten()
    
    cosine_similarities.append(cosine_similarity)
cosine_similarities = np.vstack(cosine_similarities)
# sentence_embeddings = np.concatenate([e.reshape(1, -1) for e in list(doc_embeddings.values())])
# cosine_similarities = np.dot(train_embeddings.values(), sentence_embeddings.T)
np.save(f"cosine_similarities_model_{fold}.npy", cosine_similarities)

np.save(f"sentence_embeddings_{fold}.npy", sentence_embeddings)

train_embeddings = {key: train_embeddings[key] for key in sorted(train_embeddings)}
train_embeddings = np.concatenate([e.reshape(1, -1) for e in list(train_embeddings.values())])
np.save(f"train_embeddings_{fold}.npy", train_embeddings)

with open("index_text_embeddings_index.pkl", "wb") as f:
    pickle.dump(index_text_embeddings_index, f)

Writing fold1.py


In [12]:
# !python fold1.py

In [13]:
%%writefile fold2.py
from tqdm.auto import tqdm
# from bs4 import BeautifulSoup
import gc
import pandas as pd
import pickle
import sys
import numpy as np
from tqdm.autonotebook import trange
from sklearn.model_selection import GroupKFold
import json
import torch
from numpy.linalg import norm
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel,BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
)
from peft import PeftModel
from peft import prepare_model_for_kbit_training
import json
import copy
import warnings
warnings.filterwarnings('ignore')
import os
import re

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


fold = 2
train_df = pd.read_parquet("tmp.parquet")
def batch_to_device(batch, target_device):
    """
    send a pytorch batch to a device (CPU/GPU)
    """
    for key in batch:
        if isinstance(batch[key], Tensor):
            batch[key] = batch[key].to(target_device)
    return batch

def last_token_pool(last_hidden_states: Tensor,
                    attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'
def inference(df, model, tokenizer, device):
    batch_size = 2
    max_length = 512
    sentences = list(df['query_text'].values)
    pids = list(df['order_index'].values)
    all_embeddings = []
    length_sorted_idx = np.argsort([-len(sen) for sen in sentences])
    sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
    for start_index in trange(0, len(sentences), batch_size, desc="Batches", disable=False):
        sentences_batch = sentences_sorted[start_index: start_index + batch_size]
        features = tokenizer(sentences_batch, max_length=max_length, padding=True, truncation=True,
                             return_tensors="pt")
        features = batch_to_device(features, device)
        with torch.no_grad():
            outputs = model(**features)
            embeddings = last_token_pool(outputs.last_hidden_state, features['attention_mask'])
            embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
            embeddings = embeddings.detach().cpu().numpy().tolist()
        all_embeddings.extend(embeddings)

    all_embeddings = [np.array(all_embeddings[idx]).reshape(1, -1) for idx in np.argsort(length_sorted_idx)]

    sentence_embeddings = np.concatenate(all_embeddings, axis=0)
    result = {int(pids[i]): em for i, em in enumerate(sentence_embeddings)}
    return result

path_prefix = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"
model_path = "/kaggle/input/qwen-instruct-14b"
# lora_path="/kaggle/input/aa1203-0254/hp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd001/epoch_4_model" #527
lora_path="/kaggle/input/zsq-other-fourfold-529/hp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd000_6ra_fold2/hp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd000_6ra_fold2/epoch_4_model"
# lora_path = "/kaggle/input/zsq-other-fourfold-529-wd001/rhp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd001_6ra_fold2/hp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd001_6ra_fold2/epoch_4_model"
# device='cuda'
VALID = False

misconception_mapping = pd.read_csv(f"{path_prefix}/misconception_mapping.csv")

tokenizer = AutoTokenizer.from_pretrained(model_path)
# bnb_config = BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_quant_type="nf4",
#             bnb_4bit_compute_dtype=torch.bfloat16
#         )
model = AutoModel.from_pretrained(model_path, 
                                  torch_dtype=torch.float16,
                                  # quantization_config=bnb_config,
                                  device_map='auto',
                                 )
if lora_path!="none":

    model = PeftModel.from_pretrained(model, lora_path)#.eval().to(args.device)
    # model = model.merge_and_unload()
model = model.eval()

train_embeddings = inference(train_df, model, tokenizer, 'cuda')

misconception_mapping['query_text'] = misconception_mapping['MisconceptionName']
misconception_mapping['order_index'] = misconception_mapping['MisconceptionId']
doc_embeddings = inference(misconception_mapping, model, tokenizer, 'cuda')

sentence_embeddings = np.concatenate([e.reshape(1, -1) for e in list(doc_embeddings.values())])
index_text_embeddings_index = {index: paper_id for index, paper_id in
                                         enumerate(list(doc_embeddings.keys()))}


cosine_similarities = []
for _, row in tqdm(train_df.iterrows()):
    query_id = row['order_index']
    query_em = train_embeddings[query_id].reshape(1, -1)
    
    cosine_similarity = np.dot(query_em, sentence_embeddings.T).flatten()
    
    cosine_similarities.append(cosine_similarity)
cosine_similarities = np.vstack(cosine_similarities)
# sentence_embeddings = np.concatenate([e.reshape(1, -1) for e in list(doc_embeddings.values())])
# cosine_similarities = np.dot(train_embeddings.values(), sentence_embeddings.T)
np.save(f"cosine_similarities_model_{fold}.npy", cosine_similarities)

np.save(f"sentence_embeddings_{fold}.npy", sentence_embeddings)

train_embeddings = {key: train_embeddings[key] for key in sorted(train_embeddings)}
train_embeddings = np.concatenate([e.reshape(1, -1) for e in list(train_embeddings.values())])
np.save(f"train_embeddings_{fold}.npy", train_embeddings)

Writing fold2.py


In [14]:
# !python fold2.py

In [15]:
%%writefile fold3.py
from tqdm.auto import tqdm
# from bs4 import BeautifulSoup
import gc
import pandas as pd
import pickle
import sys
import numpy as np
from tqdm.autonotebook import trange
from sklearn.model_selection import GroupKFold
import json
import torch
from numpy.linalg import norm
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel,BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
)
from peft import PeftModel
from peft import prepare_model_for_kbit_training
import json
import copy
import warnings
warnings.filterwarnings('ignore')
import os
import re

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


fold = 3
train_df = pd.read_parquet("tmp.parquet")
def batch_to_device(batch, target_device):
    """
    send a pytorch batch to a device (CPU/GPU)
    """
    for key in batch:
        if isinstance(batch[key], Tensor):
            batch[key] = batch[key].to(target_device)
    return batch

def last_token_pool(last_hidden_states: Tensor,
                    attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'
def inference(df, model, tokenizer, device):
    batch_size = 2
    max_length = 512
    sentences = list(df['query_text'].values)
    pids = list(df['order_index'].values)
    all_embeddings = []
    length_sorted_idx = np.argsort([-len(sen) for sen in sentences])
    sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
    for start_index in trange(0, len(sentences), batch_size, desc="Batches", disable=False):
        sentences_batch = sentences_sorted[start_index: start_index + batch_size]
        features = tokenizer(sentences_batch, max_length=max_length, padding=True, truncation=True,
                             return_tensors="pt")
        features = batch_to_device(features, device)
        with torch.no_grad():
            outputs = model(**features)
            embeddings = last_token_pool(outputs.last_hidden_state, features['attention_mask'])
            embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
            embeddings = embeddings.detach().cpu().numpy().tolist()
        all_embeddings.extend(embeddings)

    all_embeddings = [np.array(all_embeddings[idx]).reshape(1, -1) for idx in np.argsort(length_sorted_idx)]

    sentence_embeddings = np.concatenate(all_embeddings, axis=0)
    result = {int(pids[i]): em for i, em in enumerate(sentence_embeddings)}
    return result

path_prefix = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"
model_path = "/kaggle/input/qwen-instruct-14b"
# lora_path="/kaggle/input/aa1203-2305/hp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd000_5ra/epoch_4_model" #527
lora_path = "/kaggle/input/zsq-other-fourfold-529/hp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd000_6ra_fold3/hp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd000_6ra_fold3/epoch_4_model"
# lora_path = "/kaggle/input/zsq-other-fourfold-529-wd001/rhp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd001_6ra_fold3/hp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd001_6ra_fold3/epoch_4_model"
# device='cuda'
VALID = False

misconception_mapping = pd.read_csv(f"{path_prefix}/misconception_mapping.csv")

tokenizer = AutoTokenizer.from_pretrained(model_path)
# bnb_config = BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_quant_type="nf4",
#             bnb_4bit_compute_dtype=torch.bfloat16
#         )
model = AutoModel.from_pretrained(model_path, 
                                  torch_dtype=torch.float16,
                                  # quantization_config=bnb_config,
                                  device_map='auto',
                                 )
if lora_path!="none":

    model = PeftModel.from_pretrained(model, lora_path)#.eval().to(args.device)
    # model = model.merge_and_unload()
model = model.eval()

train_embeddings = inference(train_df, model, tokenizer, 'cuda')

misconception_mapping['query_text'] = misconception_mapping['MisconceptionName']
misconception_mapping['order_index'] = misconception_mapping['MisconceptionId']
doc_embeddings = inference(misconception_mapping, model, tokenizer, 'cuda')

sentence_embeddings = np.concatenate([e.reshape(1, -1) for e in list(doc_embeddings.values())])
index_text_embeddings_index = {index: paper_id for index, paper_id in
                                         enumerate(list(doc_embeddings.keys()))}


cosine_similarities = []
for _, row in tqdm(train_df.iterrows()):
    query_id = row['order_index']
    query_em = train_embeddings[query_id].reshape(1, -1)
    
    cosine_similarity = np.dot(query_em, sentence_embeddings.T).flatten()
    
    cosine_similarities.append(cosine_similarity)
cosine_similarities = np.vstack(cosine_similarities)
# sentence_embeddings = np.concatenate([e.reshape(1, -1) for e in list(doc_embeddings.values())])
# cosine_similarities = np.dot(train_embeddings.values(), sentence_embeddings.T)
np.save(f"cosine_similarities_model_{fold}.npy", cosine_similarities)
np.save(f"sentence_embeddings_{fold}.npy", sentence_embeddings)
train_embeddings = {key: train_embeddings[key] for key in sorted(train_embeddings)}
train_embeddings = np.concatenate([e.reshape(1, -1) for e in list(train_embeddings.values())])
np.save(f"train_embeddings_{fold}.npy", train_embeddings)

Writing fold3.py


In [16]:
# !python fold3.py

In [17]:
%%writefile fold4.py
from tqdm.auto import tqdm
# from bs4 import BeautifulSoup
import gc
import pandas as pd
import pickle
import sys
import numpy as np
from tqdm.autonotebook import trange
from sklearn.model_selection import GroupKFold
import json
import torch
from numpy.linalg import norm
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel,BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
)
from peft import PeftModel
from peft import prepare_model_for_kbit_training
import json
import copy
import warnings
warnings.filterwarnings('ignore')
import os
import re

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


fold = 4
train_df = pd.read_parquet("tmp.parquet")
def batch_to_device(batch, target_device):
    """
    send a pytorch batch to a device (CPU/GPU)
    """
    for key in batch:
        if isinstance(batch[key], Tensor):
            batch[key] = batch[key].to(target_device)
    return batch

def last_token_pool(last_hidden_states: Tensor,
                    attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'
def inference(df, model, tokenizer, device):
    batch_size = 2
    max_length = 512
    sentences = list(df['query_text'].values)
    pids = list(df['order_index'].values)
    all_embeddings = []
    length_sorted_idx = np.argsort([-len(sen) for sen in sentences])
    sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
    for start_index in trange(0, len(sentences), batch_size, desc="Batches", disable=False):
        sentences_batch = sentences_sorted[start_index: start_index + batch_size]
        features = tokenizer(sentences_batch, max_length=max_length, padding=True, truncation=True,
                             return_tensors="pt")
        features = batch_to_device(features, device)
        with torch.no_grad():
            outputs = model(**features)
            embeddings = last_token_pool(outputs.last_hidden_state, features['attention_mask'])
            embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
            embeddings = embeddings.detach().cpu().numpy().tolist()
        all_embeddings.extend(embeddings)

    all_embeddings = [np.array(all_embeddings[idx]).reshape(1, -1) for idx in np.argsort(length_sorted_idx)]

    sentence_embeddings = np.concatenate(all_embeddings, axis=0)
    result = {int(pids[i]): em for i, em in enumerate(sentence_embeddings)}
    return result

path_prefix = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"
model_path = "/kaggle/input/qwen-instruct-14b"
# lora_path="/kaggle/input/aa1204-2132/hp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd001_6ra/epoch_4_model" #526
lora_path = "/kaggle/input/zsq-other-fourfold-529/hp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd000_6ra_fold4/hp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd000_6ra_fold4/epoch_4_model"
# lora_path = "/kaggle/input/zsq-other-fourfold-529-wd001/rhp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd001_6ra_fold4/hp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd001_6ra_fold4/epoch_4_model"
# device='cuda'
VALID = False

misconception_mapping = pd.read_csv(f"{path_prefix}/misconception_mapping.csv")

tokenizer = AutoTokenizer.from_pretrained(model_path)
# bnb_config = BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_quant_type="nf4",
#             bnb_4bit_compute_dtype=torch.bfloat16
#         )
model = AutoModel.from_pretrained(model_path, 
                                  torch_dtype=torch.float16,
                                  # quantization_config=bnb_config,
                                  device_map='auto',
                                 )
if lora_path!="none":

    model = PeftModel.from_pretrained(model, lora_path)#.eval().to(args.device)
    # model = model.merge_and_unload()
model = model.eval()

train_embeddings = inference(train_df, model, tokenizer, 'cuda')

misconception_mapping['query_text'] = misconception_mapping['MisconceptionName']
misconception_mapping['order_index'] = misconception_mapping['MisconceptionId']
doc_embeddings = inference(misconception_mapping, model, tokenizer, 'cuda')

sentence_embeddings = np.concatenate([e.reshape(1, -1) for e in list(doc_embeddings.values())])
index_text_embeddings_index = {index: paper_id for index, paper_id in
                                         enumerate(list(doc_embeddings.keys()))}


cosine_similarities = []
for _, row in tqdm(train_df.iterrows()):
    query_id = row['order_index']
    query_em = train_embeddings[query_id].reshape(1, -1)
    
    cosine_similarity = np.dot(query_em, sentence_embeddings.T).flatten()
    
    cosine_similarities.append(cosine_similarity)
cosine_similarities = np.vstack(cosine_similarities)
# sentence_embeddings = np.concatenate([e.reshape(1, -1) for e in list(doc_embeddings.values())])
# cosine_similarities = np.dot(train_embeddings.values(), sentence_embeddings.T)
np.save(f"cosine_similarities_model_{fold}.npy", cosine_similarities)
np.save(f"sentence_embeddings_{fold}.npy", sentence_embeddings)
train_embeddings = {key: train_embeddings[key] for key in sorted(train_embeddings)}
train_embeddings = np.concatenate([e.reshape(1, -1) for e in list(train_embeddings.values())])
np.save(f"train_embeddings_{fold}.npy", train_embeddings)

Writing fold4.py


In [18]:
# !python fold4.py

In [19]:
%%writefile fold5.py
from tqdm.auto import tqdm
# from bs4 import BeautifulSoup
import gc
import pandas as pd
import pickle
import sys
import numpy as np
from tqdm.autonotebook import trange
from sklearn.model_selection import GroupKFold
import json
import torch
from numpy.linalg import norm
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel,BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
)
from peft import PeftModel
from peft import prepare_model_for_kbit_training
import json
import copy
import warnings
warnings.filterwarnings('ignore')
import os
import re

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


fold = 5
train_df = pd.read_parquet("tmp.parquet")
def batch_to_device(batch, target_device):
    """
    send a pytorch batch to a device (CPU/GPU)
    """
    for key in batch:
        if isinstance(batch[key], Tensor):
            batch[key] = batch[key].to(target_device)
    return batch

def last_token_pool(last_hidden_states: Tensor,
                    attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'
def inference(df, model, tokenizer, device):
    batch_size = 2
    max_length = 512
    sentences = list(df['query_text'].values)
    pids = list(df['order_index'].values)
    all_embeddings = []
    length_sorted_idx = np.argsort([-len(sen) for sen in sentences])
    sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
    for start_index in trange(0, len(sentences), batch_size, desc="Batches", disable=False):
        sentences_batch = sentences_sorted[start_index: start_index + batch_size]
        features = tokenizer(sentences_batch, max_length=max_length, padding=True, truncation=True,
                             return_tensors="pt")
        features = batch_to_device(features, device)
        with torch.no_grad():
            outputs = model(**features)
            embeddings = last_token_pool(outputs.last_hidden_state, features['attention_mask'])
            embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
            embeddings = embeddings.detach().cpu().numpy().tolist()
        all_embeddings.extend(embeddings)

    all_embeddings = [np.array(all_embeddings[idx]).reshape(1, -1) for idx in np.argsort(length_sorted_idx)]

    sentence_embeddings = np.concatenate(all_embeddings, axis=0)
    result = {int(pids[i]): em for i, em in enumerate(sentence_embeddings)}
    return result

path_prefix = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"
model_path = "/kaggle/input/qwen-instruct-14b"
# lora_path="/kaggle/input/v36-filterv3-rationale/epoch_19_model" #503
lora_path = "/kaggle/input/zsq-other-fourfold-529/hp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd000_6ra_fold5/hp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd000_6ra_fold5/epoch_4_model"
# lora_path = "/kaggle/input/zsq-other-fourfold-529-wd001/rhp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd001_6ra_fold5/hp_code_64_128_epoch5_tmp002_lr1e4_gd1_wd001_6ra_fold5/epoch_4_model"
# device='cuda'
VALID = False

misconception_mapping = pd.read_csv(f"{path_prefix}/misconception_mapping.csv")

tokenizer = AutoTokenizer.from_pretrained(model_path)
# bnb_config = BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_quant_type="nf4",
#             bnb_4bit_compute_dtype=torch.bfloat16
#         )
model = AutoModel.from_pretrained(model_path, 
                                  torch_dtype=torch.float16,
                                  # quantization_config=bnb_config,
                                  device_map='auto',
                                 )
if lora_path!="none":

    model = PeftModel.from_pretrained(model, lora_path)#.eval().to(args.device)
    # model = model.merge_and_unload()
model = model.eval()

train_embeddings = inference(train_df, model, tokenizer, 'cuda')

misconception_mapping['query_text'] = misconception_mapping['MisconceptionName']
misconception_mapping['order_index'] = misconception_mapping['MisconceptionId']
doc_embeddings = inference(misconception_mapping, model, tokenizer, 'cuda')

sentence_embeddings = np.concatenate([e.reshape(1, -1) for e in list(doc_embeddings.values())])
index_text_embeddings_index = {index: paper_id for index, paper_id in
                                         enumerate(list(doc_embeddings.keys()))}


cosine_similarities = []
for _, row in tqdm(train_df.iterrows()):
    query_id = row['order_index']
    query_em = train_embeddings[query_id].reshape(1, -1)
    
    cosine_similarity = np.dot(query_em, sentence_embeddings.T).flatten()
    
    cosine_similarities.append(cosine_similarity)
cosine_similarities = np.vstack(cosine_similarities)
# sentence_embeddings = np.concatenate([e.reshape(1, -1) for e in list(doc_embeddings.values())])
# cosine_similarities = np.dot(train_embeddings.values(), sentence_embeddings.T)
np.save(f"cosine_similarities_model_{fold}.npy", cosine_similarities)
np.save(f"sentence_embeddings_{fold}.npy", sentence_embeddings)
train_embeddings = {key: train_embeddings[key] for key in sorted(train_embeddings)}
train_embeddings = np.concatenate([e.reshape(1, -1) for e in list(train_embeddings.values())])
np.save(f"train_embeddings_{fold}.npy", train_embeddings)

Writing fold5.py


In [20]:
# !python fold5.py

In [21]:
%%writefile fold6.py
from tqdm.auto import tqdm
# from bs4 import BeautifulSoup
import gc
import pandas as pd
import pickle
import sys
import numpy as np
from tqdm.autonotebook import trange
from sklearn.model_selection import GroupKFold
import json
import torch
from numpy.linalg import norm
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel,BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
)
from peft import PeftModel
from peft import prepare_model_for_kbit_training
import json
import copy
import warnings
warnings.filterwarnings('ignore')
import os
import re

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


fold = 6
train_df = pd.read_parquet("tmp.parquet")
def batch_to_device(batch, target_device):
    """
    send a pytorch batch to a device (CPU/GPU)
    """
    for key in batch:
        if isinstance(batch[key], Tensor):
            batch[key] = batch[key].to(target_device)
    return batch

def last_token_pool(last_hidden_states: Tensor,
                    attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'
def inference(df, model, tokenizer, device):
    batch_size = 2
    max_length = 512
    sentences = list(df['query_text'].values)
    pids = list(df['order_index'].values)
    all_embeddings = []
    length_sorted_idx = np.argsort([-len(sen) for sen in sentences])
    sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
    for start_index in trange(0, len(sentences), batch_size, desc="Batches", disable=False):
        sentences_batch = sentences_sorted[start_index: start_index + batch_size]
        features = tokenizer(sentences_batch, max_length=max_length, padding=True, truncation=True,
                             return_tensors="pt")
        features = batch_to_device(features, device)
        with torch.no_grad():
            outputs = model(**features)
            embeddings = last_token_pool(outputs.last_hidden_state, features['attention_mask'])
            embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
            embeddings = embeddings.detach().cpu().numpy().tolist()
        all_embeddings.extend(embeddings)

    all_embeddings = [np.array(all_embeddings[idx]).reshape(1, -1) for idx in np.argsort(length_sorted_idx)]

    sentence_embeddings = np.concatenate(all_embeddings, axis=0)
    result = {int(pids[i]): em for i, em in enumerate(sentence_embeddings)}
    return result

path_prefix = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"
model_path = "/kaggle/input/qwen-instruct-14b"
# lora_path="/kaggle/input/v36-filterv3-rationale/epoch_19_model" #503
lora_path = "/kaggle/input/hp-all-lora-ensemble/hp_all_lora_ensemble"
# lora_path = "/kaggle/input/v8v13v14-19/v8v13v14_19/lora_merge.bin"
# device='cuda'
VALID = False

misconception_mapping = pd.read_csv(f"{path_prefix}/misconception_mapping.csv")

tokenizer = AutoTokenizer.from_pretrained(model_path)
# bnb_config = BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_quant_type="nf4",
#             bnb_4bit_compute_dtype=torch.bfloat16
#         )
model = AutoModel.from_pretrained(model_path, 
                                  torch_dtype=torch.float16,
                                  # quantization_config=bnb_config,
                                  device_map='auto',
                                 )
if lora_path!="none":

    model = PeftModel.from_pretrained(model, lora_path)#.eval().to(args.device)
    # model = model.merge_and_unload()
model = model.eval()

train_embeddings = inference(train_df, model, tokenizer, 'cuda')

misconception_mapping['query_text'] = misconception_mapping['MisconceptionName']
misconception_mapping['order_index'] = misconception_mapping['MisconceptionId']
doc_embeddings = inference(misconception_mapping, model, tokenizer, 'cuda')

sentence_embeddings = np.concatenate([e.reshape(1, -1) for e in list(doc_embeddings.values())])
index_text_embeddings_index = {index: paper_id for index, paper_id in
                                         enumerate(list(doc_embeddings.keys()))}


cosine_similarities = []
for _, row in tqdm(train_df.iterrows()):
    query_id = row['order_index']
    query_em = train_embeddings[query_id].reshape(1, -1)
    
    cosine_similarity = np.dot(query_em, sentence_embeddings.T).flatten()
    
    cosine_similarities.append(cosine_similarity)
cosine_similarities = np.vstack(cosine_similarities)
# sentence_embeddings = np.concatenate([e.reshape(1, -1) for e in list(doc_embeddings.values())])
# cosine_similarities = np.dot(train_embeddings.values(), sentence_embeddings.T)
np.save(f"cosine_similarities_model_{fold}.npy", cosine_similarities)
np.save(f"sentence_embeddings_{fold}.npy", sentence_embeddings)
train_embeddings = {key: train_embeddings[key] for key in sorted(train_embeddings)}
train_embeddings = np.concatenate([e.reshape(1, -1) for e in list(train_embeddings.values())])
np.save(f"train_embeddings_{fold}.npy", train_embeddings)

with open("index_text_embeddings_index.pkl", "wb") as f:
    pickle.dump(index_text_embeddings_index, f)

Writing fold6.py


In [22]:
!python fold6.py

Batches: 100%|██████████████████████████████| 1294/1294 [03:39<00:00,  5.89it/s]
9it [00:00, 223.68it/s]


In [23]:
# #result 1
# with open("index_text_embeddings_index.pkl", "rb") as f:
#     index_text_embeddings_index = pickle.load(f)

# all_cosine_similarities = []
# for i in range(1,6):
#     cosine_similarities = np.load(f"cosine_similarities_model_{i}.npy")
#     all_cosine_similarities.append(cosine_similarities)

# # 融合相似度
# combined_cosine_similarities = np.mean(all_cosine_similarities, axis=0)

# predicts_test = []
# for row in tqdm(combined_cosine_similarities):
    
#     sort_index = np.argsort(-row)[:25]
#     pids = [index_text_embeddings_index[index] for index in sort_index]
#     predicts_test.append(pids)

In [24]:
#result 2
with open("index_text_embeddings_index.pkl", "rb") as f:
    index_text_embeddings_index = pickle.load(f)

sentence_embeddings = []
train_embeddings = []

for i in range(6,7):
    sentence_embedding = np.load(f"sentence_embeddings_{i}.npy")
    train_embedding = np.load(f"train_embeddings_{i}.npy")
    
    sentence_embeddings.append(sentence_embedding)
    train_embeddings.append(train_embedding)

sentence_embeddings = np.mean(sentence_embeddings, axis=0)
train_embeddings = np.mean(train_embeddings, axis=0)

combined_cosine_similarities = np.dot(train_embeddings,sentence_embeddings.T)
predicts_test_2 = []
for row in tqdm(combined_cosine_similarities):
    
    sort_index = np.argsort(-row)[:25]
    pids = [index_text_embeddings_index[index] for index in sort_index]
    predicts_test_2.append(pids)



  0%|          | 0/9 [00:00<?, ?it/s]

In [25]:
# # rank 融合

# rank_predict = []
# alpha = 1

# for p in zip(predicts_test,predicts_test_2):
#     score = {}
#     for x in p:
#         for i,k in enumerate(x):
#             if k not in score:
#                 score[k] = 1/(i+alpha)
#             else:
#                 score[k] += 1/(i+alpha)
#     print(p[0])
#     print(p[1])
#     score = sorted(score, key=score.get, reverse=True)
#     rank_predict.append(score)
#     print(score)
#     print("===="*20)

In [26]:
VALID = False

if VALID:
    train_df['recall_ids'] = predicts_test_2
    print(mapk([[data] for data in train_df['answer_id'].values],train_df['recall_ids'].values))
else:
    train_df['MisconceptionId'] = [' '.join(map(str,c)) for c in predicts_test_2]
    # sub = []
    # for _,row in train_df.iterrows():
    #     sub.append(
    #         {
    #             "QuestionId_Answer":f"{row['QuestionId']}_{row['answer_name']}",
    #             "MisconceptionId":row['MisconceptionId']
    #         }
    #     )
    # submission_df = pd.DataFrame(sub)
    # submission_df.to_csv("submission.csv", index=False)
    # print("Submission file created successfully!")
    train_df.to_parquet("recall.parquet", index=False)

In [27]:
%reset -f
import gc
gc.collect()

0

# Rank

In [28]:
# # rank 推理
# from tqdm.auto import tqdm
# # from bs4 import BeautifulSoup
# import gc
# import pandas as pd
# import pickle
# import sys
# import numpy as np
# from tqdm.autonotebook import trange
# from sklearn.model_selection import GroupKFold
# import json
# import torch
# from numpy.linalg import norm
# import torch.nn.functional as F
# from torch import Tensor
# from transformers import AutoTokenizer, AutoModel,BitsAndBytesConfig
# from peft import (
#     LoraConfig,
#     get_peft_model,
# )

# from peft import PeftModel
# import json
# import copy
# import warnings
# warnings.filterwarnings('ignore')
# import os
# from transformers import AutoModelForCausalLM,AutoModelForSequenceClassification


# def batch_to_device(batch, target_device):
#     """
#     send a pytorch batch to a device (CPU/GPU)
#     """
#     for key in batch:
#         if isinstance(batch[key], Tensor):
#             batch[key] = batch[key].to(target_device)
#     return batch
    
# def create_one_example(tokenier,query, doc):
#     query_max_len = 512
    
#     qry_inputs = tokenizer.encode(query, truncation=True, max_length = query_max_len, add_special_tokens=False)
#     doc_inputs = tokenizer.encode(doc, add_special_tokens=False)
#     item = tokenizer.prepare_for_model(
#         qry_inputs,
#         doc_inputs
#     )
#     # print(item)
#     # print(tokenizer.decode(item['input_ids']))
#     return item

# def inference(df,model,tokenizer,prompt,device):
    
    
#     #yes_loc = tokenizer('Yes', add_special_tokens=False)['input_ids'][-1]
#     batch_size = 2

#     pids = list(range(len(df)))
#     query = [prompt + q + '\n### Explanlation:' for q in df['query'].values]
#     doc = list(df['MisconceptionName'])
#     sentences = [create_one_example(tokenizer,q,d) for q,d in zip(query,doc)]
    
#     all_scores = []
#     length_sorted_idx = np.argsort([-len(sen['input_ids']) for sen in sentences])
#     sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
#     for start_index in trange(0, len(sentences), batch_size, desc="Batches", disable=False):
#         sentences_batch = sentences_sorted[start_index: start_index + batch_size]
#         features = tokenizer.pad(sentences_batch,
#                                 padding=True,
#                                 max_length= 576, # 512+64
#                                 return_tensors='pt')
        
#         print(features['input_ids'].shape)
#         features = batch_to_device(features, device)
#         with torch.no_grad():
#             outputs = model(**features)
#             scores = outputs.logits[:, -1]
#             scores = scores.detach().cpu().numpy().tolist()
#         all_scores.extend(scores)   
        
#     all_scores = [all_scores[idx] for idx in np.argsort(length_sorted_idx)]
#     df['score'] = all_scores
#     return df


# path_prefix = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"
# model_path = "/kaggle/input/qwen-instruct-14b"
# lora_path="/kaggle/input/rank-all-lora-ensemble-v2/rank-all-lora-ensemble-v2"



# tokenizer = AutoTokenizer.from_pretrained(model_path)
# tokenizer.padding_side = 'left'
# # bnb_config = BitsAndBytesConfig(
# #             load_in_4bit=True,
# #             bnb_4bit_use_double_quant=True,
# #             bnb_4bit_quant_type="nf4",
# #             bnb_4bit_compute_dtype=torch.float16
# #         )
# model = AutoModelForSequenceClassification.from_pretrained(model_path, 
#                                   #quantization_config=bnb_config,
#                                   torch_dtype=torch.float16,
#                                   device_map='auto',
#                                   num_labels = 1
#                                  )
# if lora_path!="none":
#     #model = get_peft_model(model, config)
#     #d = torch.load(lora_path, map_location=model.device)
#     #model.load_state_dict(d, strict=False)
#     model = PeftModel.from_pretrained(model, lora_path)#.eval().to(args.device)
#     model = model.merge_and_unload()
    
# model = model.eval()

# model.config.pad_token_id = tokenizer.pad_token_id




# prompt = "Given a query containing a math problem and an incorrect answer, along with an explanation of the mistake made in the answer, determine whether the explanation accurately summarizes the error in the answer by providing a prediction of either 'Yes' or 'No'. \n Query:"
# def get_query(row):
#     real_answer_id = row['CorrectAnswer']
#     real_text = row[f'Answer{real_answer_id}Text']
#     c = row['answer_name']
#     query_text = f"### SubjectName: {row['SubjectName']}\n### ConstructName: {row['ConstructName']}\n### Question: {row['QuestionText']}\n### Correct Answer: {real_text}\n### Misconcepte Incorrect answer: {row[f'Answer{c}Text']}"
#     return query_text
    
# train_df = pd.read_parquet('recall.parquet')
# misconception_mapping = pd.read_csv(f"{path_prefix}/misconception_mapping.csv")

# train_df['order_index'] = list(range(len(train_df)))
# train_df['query'] = train_df.apply(get_query,axis = 1)
# # add
# train_df['query'] = train_df['query'] + "\n### Rationale: " + train_df['rationale']

# train_df['MisconceptionId'] = train_df['MisconceptionId'].apply(lambda x:[int(i) for i in x.split(' ')])


# ## 只取前15
# train_df['MisconceptionId_15_other'] = train_df['MisconceptionId'].apply(lambda x: x[15:])
# train_df['MisconceptionId'] = train_df['MisconceptionId'].apply(lambda x: x[:15])

# train_explode = train_df.explode('MisconceptionId')
# train_explode = pd.merge(train_explode,misconception_mapping,on = 'MisconceptionId',how = 'left')

# train_explode = inference(train_explode,model,tokenizer,prompt,'cuda')

# train_explode = train_explode.groupby('order_index',group_keys=False).apply(lambda x: x.sort_values('score',ascending=False))
# tmp_res = train_explode.groupby('order_index').agg({'MisconceptionId': list}).reset_index()
# train_df = pd.merge(train_df.drop('MisconceptionId',axis = 1),tmp_res,on = 'order_index',how = 'left')


# ## 合并
# train_df['MisconceptionId'] = train_df.apply(lambda row: row['MisconceptionId']+row['MisconceptionId_15_other'], axis=1)

# train_df['MisconceptionId'] = train_df['MisconceptionId'].apply(lambda x:' '.join([str(i) for i in x]))
# train_df["QuestionId_Answer"] = train_df.apply(lambda x:f'{x["QuestionId"]}_{x["answer_name"]}',axis = 1)
# train_df[['QuestionId_Answer','MisconceptionId']].to_csv("submission.csv", index=False)
# print("Submission file created successfully!")


In [29]:
# train_df

# 32b vllm

In [30]:
%%writefile test.py
# rank 推理
from tqdm.auto import tqdm
# from bs4 import BeautifulSoup
import gc
import pandas as pd
import pickle
import sys
import numpy as np
from tqdm.autonotebook import trange
from sklearn.model_selection import GroupKFold
import json
import torch
from numpy.linalg import norm
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel,BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
)

from peft import PeftModel
import json
import copy
import warnings
warnings.filterwarnings('ignore')
import os
from transformers import AutoModelForCausalLM,AutoModelForSequenceClassification

import vllm
from typing import Any, Dict, List
from transformers import LogitsProcessor
import torch
from time import time
#os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "ray"
#os.environ['']


path_prefix = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"
model_path = '/kaggle/input/32b-new-last-66-58-awq1212/32b-new-last-66-58-AWQ—1212'

prompt = "Given a query containing a math problem and an incorrect answer, along with an explanation of the mistake made in the answer, determine whether the explanation accurately summarizes the error in the answer by providing a prediction of either 'Yes' or 'No'. \n Query:"



llm = vllm.LLM(
    model_path,
    quantization="awq",
    tensor_parallel_size=2,
    gpu_memory_utilization=0.90, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=5120,
    disable_log_stats=True,
    distributed_executor_backend="ray"
)

def vllm_infer(df,model):

    pids = list(range(len(df)))
    query = [prompt + q + '\n### Explanlation:' for q in df['query'].values]
    doc = list(df['MisconceptionName'])
    sentences = [q+d+'\n' for q,d in zip(query,doc)]
    #print(sentences[0])
    tokenizer = llm.get_tokenizer()
    
    choices = ["Yes"]

    KEEP = []
    for x in choices:
        c = tokenizer.encode(x,add_special_tokens=False)[0]
        KEEP.append(c)

    class DigitLogitsProcessor(LogitsProcessor):
        def __init__(self, tokenizer):
            self.allowed_ids = KEEP

        def __call__(self, input_ids: List[int], scores: torch.Tensor) -> torch.Tensor:
            scores[self.allowed_ids] += 1
            return scores

    start = time()

    logits_processors = [DigitLogitsProcessor(tokenizer)]
    responses = llm.generate(
        sentences,
        vllm.SamplingParams(
            n=1,  # Number of output sequences to return for each prompt.
            top_p=0.9,  # Float that controls the cumulative probability of the top tokens to consider.
            temperature=0,  # randomness of the sampling
            seed=777, # Seed for reprodicibility
            skip_special_tokens=True,  # Whether to skip special tokens in the output.
            max_tokens=1,  # Maximum number of tokens to generate per output sequence.
            logits_processors=logits_processors,
            logprobs = 5
        ),
        use_tqdm = True
    )

    end = time()
    elapsed = (end-start)/60. #minutes
    print(f"Inference took {elapsed} minutes!")
    
    results = []
    errors = 0
    score = []
    for i,response in enumerate(responses):
        try:
            x = response.outputs[0].logprobs[0]
            logprobs = []
            for k in KEEP:
                if k in x:
                    score.append(x[k].logprob)
                else:
                    score.append(-100)
                    #print(f"bad logits {i}")
        except:
            #print(f"error {i}")
            results.append(-100)
            errors += 1

    print(f"There were {errors} inference errors out of {i+1} inferences")
    df['score'] = score
    return df

def get_query(row):
    real_answer_id = row['CorrectAnswer']
    real_text = row[f'Answer{real_answer_id}Text']
    c = row['answer_name']
    query_text = f"### SubjectName: {row['SubjectName']}\n### ConstructName: {row['ConstructName']}\n### Question: {row['QuestionText']}\n### Correct Answer: {real_text}\n### Misconcepte Incorrect answer: {row[f'Answer{c}Text']}"
    return query_text
    
train_df = pd.read_parquet('recall.parquet')
misconception_mapping = pd.read_csv(f"{path_prefix}/misconception_mapping.csv")

train_df['order_index'] = list(range(len(train_df)))
train_df['query'] = train_df.apply(get_query,axis = 1)
# add
train_df['query'] = train_df['query'] + "\n### Rationale: " + train_df['rationale']

train_df['MisconceptionId'] = train_df['MisconceptionId'].apply(lambda x:[int(i) for i in x.split(' ')])


## 只取前15
train_df['MisconceptionId_15_other'] = train_df['MisconceptionId'].apply(lambda x: x[20:])
train_df['MisconceptionId'] = train_df['MisconceptionId'].apply(lambda x: x[:20])

train_explode = train_df.explode('MisconceptionId')
train_explode = pd.merge(train_explode,misconception_mapping,on = 'MisconceptionId',how = 'left')

train_explode = vllm_infer(train_explode,llm)

train_explode = train_explode.groupby('order_index',group_keys=False).apply(lambda x: x.sort_values('score',ascending=False))
tmp_res = train_explode.groupby('order_index').agg({'MisconceptionId': list}).reset_index()
train_df = pd.merge(train_df.drop('MisconceptionId',axis = 1),tmp_res,on = 'order_index',how = 'left')


## 合并
train_df['MisconceptionId'] = train_df.apply(lambda row: row['MisconceptionId']+row['MisconceptionId_15_other'], axis=1)

train_df['MisconceptionId'] = train_df['MisconceptionId'].apply(lambda x:' '.join([str(i) for i in x]))
train_df["QuestionId_Answer"] = train_df.apply(lambda x:f'{x["QuestionId"]}_{x["answer_name"]}',axis = 1)
train_df[['QuestionId_Answer','MisconceptionId']].to_csv("submission.csv", index=False)
print("Submission file created successfully!")

Writing test.py


In [31]:
!python test.py

WARNING 12-12 11:34:14 config.py:246] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
2024-12-12 11:34:17,147	INFO worker.py:1749 -- Started a local Ray instance.
INFO 12-12 11:34:18 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='/kaggle/input/32b-new-last-66-58-awq1212/32b-new-last-66-58-AWQ—1212', speculative_config=None, tokenizer='/kaggle/input/32b-new-last-66-58-awq1212/32b-new-last-66-58-AWQ—1212', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=5120, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_c

In [32]:
import pandas as pd
data = pd.read_csv('submission.csv')
data

,QuestionId_Answer,MisconceptionId
0,1869_B,1345 315 2488 706 2306 1005 1507 2181 1963 251...
1,1869_C,1345 2488 2518 315 1507 2306 706 1005 2181 196...
2,1869_D,1507 2532 1672 1005 328 1392 2488 2518 1345 31...
3,1870_A,1755 59 2142 2068 167 891 1535 418 1191 1540 1...
4,1870_B,143 59 891 167 1755 885 2398 1540 363 2078 187...
5,1870_C,1755 59 2142 167 2068 891 1535 1191 418 143 68...
6,1871_A,1287 1073 2439 1059 2471 365 1665 2551 1306 12...
7,1871_C,1287 1073 1059 2439 632 365 2551 1665 1521 120...
8,1871_D,1073 1059 1287 2439 2165 365 2471 2551 2151 39...
